In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv(r"G:\work\logistica\info\input\1902info_L\cha\1902cha_jingdongwuliu.csv",sep = None,encoding='utf8')

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data.shape

(295898, 11)

In [4]:
data.head()

,"﻿""ORDER_ID""",MAIL_NO,ORDER_STATUS,PROV_NAME,CITY_NAME,CHANNEL_ID,LOGISTICS_COMPANY,ACTION,PARTNER_ACTION,ROUTE_TIME,DESCRIPTION
0,600000000000008319020100865032,V050315123412,交易完成,北京,北京,20001.0,jingdongwuliu,NaN,快递签收,2019-02-01 12:43:33.000000000,货物已交付京东物流
1,600000000000008319020100865032,V050315123412,交易完成,北京,北京,20001.0,jingdongwuliu,NaN,配送员完成揽收,2019-02-01 12:43:33.000000000,配送员高如霞已经揽收完成
2,600000000000008319020100865032,V050315123412,交易完成,北京,北京,20001.0,jingdongwuliu,NaN,分拣中心验货,2019-02-01 19:28:38.000000000,货物已到达【北京通州分拣中心】
3,600000000000008319020100865032,V050315123412,交易完成,北京,北京,20001.0,jingdongwuliu,NaN,分拣中心发货,2019-02-02 03:38:47.000000000,货物已完成分拣，离开【北京通州分拣中心】
4,600000000000008319020100865032,V050315123412,交易完成,北京,北京,20001.0,jingdongwuliu,NaN,站点收货,2019-02-02 07:55:54.000000000,货物已到达【北京纪家庙营业部】


In [5]:
df = data.copy()

In [6]:
df["PARTNER_ACTION"].value_counts()

分拣中心发货      58959
分拣中心验货      45022
配送员收货       38983
配送员完成揽收     32400
站点验货        30815
快递签收        30684
妥投          27181
站点收货        18287
再投           9023
提货登记         2059
换单打印          869
拒收            697
自提上架          475
揽件再取          305
超时催提           82
上门接货退货完成       37
逆向分拣中心发货        8
退货运单打印          6
终止揽收            3
下单取消            3
Name: PARTNER_ACTION, dtype: int64

In [7]:
df1 = df[df["PARTNER_ACTION"] == "站点收货"].reset_index(drop=True)
df2 = df[df["PARTNER_ACTION"] == "配送员收货"].reset_index(drop=True)

In [8]:
print("站点收货："+ str(df1.shape))
print("配送员收货："+ str(df2.shape))

站点收货：(18287, 11)
配送员收货：(38983, 11)


In [9]:
df2["DESCRIPTION"][0:3]

0     配送员开始配送，请您准备收货，配送员，程波，手机号，18514400720
1     配送员开始配送，请您准备收货，配送员，卢萍，手机号，17732609044
2    配送员开始配送，请您准备收货，配送员，张殿峰，手机号，17701209086
Name: DESCRIPTION, dtype: object

In [10]:
def get_JD19_lastStations(JD):
    JD_name_list = []
    for i in JD:
        JD_name_dict = {}
        JD_name_dict["JD_lastStation"] = str(re.findall("货物已到达【(\\w+)】",i)).replace("['","").replace("']","")

        JD_name_list.append(JD_name_dict)
#         print(JD_name_dict["JD_lastStation"])
    return JD_name_list

In [11]:
def get_JD19_deliveryman(JD):
    JD_name_list = []
    for i in JD:
        JD_name_dict = {}
        JD_name_dict["JD_deliveryman_name"] = str(re.findall("配送员，(\\w+)",i)).replace("['","").replace("']","")
        JD_name_dict["JD_deliveryman_tel"] = str(re.findall("手机号，(\\d+)",i)).replace("['","").replace("']","")

        JD_name_list.append(JD_name_dict)
#         print(JD_name_dict["JD_lastStation"])
    return JD_name_list

In [12]:
df3 = pd.DataFrame(get_JD19_lastStations(df1["DESCRIPTION"]))

In [13]:
df4 = pd.DataFrame(get_JD19_deliveryman(df2["DESCRIPTION"]))

In [18]:
df_LS = pd.concat([df1,df3],axis=1)
df_DM = pd.concat([df2,df4],axis=1)

In [26]:
print("df_LS："+ str(df_LS.shape))
print("df_DM："+ str(df_DM.shape))

df_LS：(18287, 12)
df_DM：(38983, 13)


In [24]:
df_LS.to_csv(r"G:\work\logistica\info\output\CHA\CSV\201902_JDWL_LS_cha.csv",index=False)
df_DM.to_csv(r"G:\work\logistica\info\output\CHA\CSV\201902_JDWL_DM_cha.csv",index=False)

In [36]:
df5 = pd.merge(df_LS,
               df_DM[['MAIL_NO','PARTNER_ACTION','ROUTE_TIME','DESCRIPTION','JD_deliveryman_name','JD_deliveryman_tel']],
               how='inner',on="MAIL_NO",suffixes=("_LS","_DM"))
df5.shape

(24915, 17)

In [62]:
df5.to_excel(r"G:\work\logistica\info\output\CHA\CSV\201902_JDWL_cha.xlsx",index=False)

In [55]:
df6 = df5[df5["PROV_NAME"] == "重庆"]

In [60]:
df6.shape

(99, 17)

In [61]:
df6['JD_deliveryman_name'].groupby(df6['JD_lastStation']).count()

JD_lastStation
三门峡卢氏营业部       1
上海华新营业部        1
上海南浦营业部        1
上海卢湾营业部        2
上海新顺营业部        1
上海金泽营业部        1
东莞大京九营业部       1
义乌后宅营业部        1
伊犁师范营业部        1
保定中天营业部        1
保定仙颐园营业部       1
兰州红山营业部        2
北京天宫院营业部       1
北京望京营业部        1
北京黄庄营业部        1
南京海峡营业部        2
南京湖熟营业部        1
厦门赵岗营业部        2
咸阳秦都营业部        2
哈尔滨盛平营业部       1
喀什莎车营业部        1
安顺平坝营业部        1
宣城旌德营业部        1
巴州库尔勒营业部       3
广州大沙头营业部       1
广州西华营业部        1
广州钟村营业部        4
广州黄阁营业部        2
廊坊九州乡营业部       1
开封兰考营业部        1
              ..
杭州致远营业部        3
汕尾海丰营业部        2
沙市营业部          1
河源源城营业部        1
泰州堰南营业部        1
洛阳大唐营业部        1
海口蓝天营业部        3
海宁钱塘营业部        1
涿州劳动关系学院京东派    1
深圳蛇口营业部        2
温岭迎宾营业部        2
温州永强营业部        1
温州白象营业部        2
温州苍南营业部        1
眉山仁寿营业部        2
石家庄南三条营业部      1
绍兴福全营业部        2
绵阳安县营业部        1
自贡荣县营业部        4
苏州大学北区加盟京东派    1
苏州新康营业部        1
贵港城东营业部        1
贵港平南营业部        1
贵阳开阳营业部        1
贵阳沙南营业部        1
遂宁射洪营业部        1
银川兴庆营业部        1

In [47]:
# # df5[df5.duplicated(subset='MAIL_NO') == True]
# df5.duplicated(subset='MAIL_NO').value_counts()



# # df6 = df[df["PARTNER_ACTION"] == "妥投"].reset_index(drop=True)

# # df6.shape

# # df7 = pd.merge(df5,df6,how='inner',on='MAIL_NO')

# # df7.shape

# # df7.duplicated(subset='MAIL_NO').value_counts()

# # df7[df7.duplicated(subset='MAIL_NO') == True]

False    16471
True      8444
dtype: int64